In [2]:
import math
import numpy as np
import scipy.interpolate

# Características de asa finita:

Essas são as características de uma asa não ideal. Elas são obtidas através do perfil.

Aqui as características do perfil serão vistas como já definidas.
Em especial, temos que destacar o coeficiente angular do perfil $a_0$ 
$$ a_0 = \frac{\Delta c_l}{\Delta \alpha} $$



## Asas de alto alongamento (AR>4)

Se AR>4:

Deve ser calcullado o fator de arrasto induzido $\delta$

$\delta(\lambda,AR) $

In [40]:
def build_delta_lut(file, skip_header=True):
    lut = np.genfromtxt(file, delimiter='\t', dtype=float, skip_header=skip_header) #TO DO: use memoization?
    return lut

delta_lut = build_delta_lut('deltaLUT.tsv')
delta_lut

array([[0.0815, 0.0946, 0.1077, ..., 0.1252, 0.1414, 0.1575],
       [0.0767, 0.0889, 0.1011, ..., 0.1174, 0.1344, 0.1513],
       [0.0705, 0.0825, 0.0944, ..., 0.1093, 0.1274, 0.1455],
       ...,
       [0.0361, 0.0504, 0.0647, ..., 0.094 , 0.1041, 0.1142],
       [0.0367, 0.0511, 0.0654, ..., 0.0948, 0.105 , 0.1152],
       [0.037 , 0.0517, 0.0663, ..., 0.0958, 0.1061, 0.1164]])

Para uma melhor visualização

In [41]:
import pandas as pd

table = pd.DataFrame(delta_lut, columns=['AR4', 'AR5', 'AR6', 'AR7', 'AR8', 'AR9', 'AR10'])
tr = np.arange(0, 1+0.005, 0.005)
table.insert(0, "tr", tr)

display(df[0:15])

,tr,AR4,AR5,AR6,AR7,AR8,AR9,AR10
0,0.000,0.0815,0.0946,0.1077,0.1165,0.1252,0.1414,0.1575
1,0.005,0.0767,0.0889,0.1011,0.1093,0.1174,0.1344,0.1513
2,0.010,0.0705,0.0825,0.0944,0.1019,0.1093,0.1274,0.1455
3,0.015,0.0627,0.0761,0.0894,0.0961,0.1028,0.1215,0.1401
4,0.020,0.0556,0.0692,0.0827,0.0900,0.0973,0.1161,0.1348
5,0.025,0.0514,0.0643,0.0771,0.0842,0.0912,0.1105,0.1297
6,0.030,0.0472,0.0597,0.0721,0.0793,0.0865,0.1041,0.1217
7,0.035,0.0437,0.0556,0.0674,0.0747,0.0820,0.0992,0.1163
8,0.040,0.0402,0.0516,0.0629,0.0701,0.0773,0.0946,0.1118
9,0.045,0.0367,0.0474,0.0581,0.0653,0.0724,0.0897,0.1070


In [4]:
def build_delta_interpolated_fuction(delta_lut):
    trs = np.arange(0, 1+0.005, 0.005)
    ars = np.arange(4,10+1, 1)
    delta_interpolated_fuction = scipy.interpolate.interp2d(ars, trs, delta_lut, kind='linear')
    
    def delta(tr, AR):
        if tr < 0 or tr > 1:
            raise ValueError("tr must be between 0 and 1")
        if AR < 4 or AR > 10:
            raise ValueError("AR must be between 4 and 10")
        return float(delta_interpolated_fuction(AR, tr))
    return delta

delta = build_delta_interpolated_fuction(delta_lut)

delta(0.1, 4)

0.017

Fator de eficiencia de Oswald

$e = \dfrac {1} {1+ \delta}$

In [5]:
def e(tr, AR):
    return 1/(1+ delta(tr, AR) )

$a = \dfrac{a_0}{1+ (a_0 / \pi \cdot e \cdot AR)}$

In [6]:
def high_AR_a(AR, a_0, tr):
    return a_0 /\
        (1 +\
            (
                a_0/(math.pi * e(tr,AR) * AR)
            )
        )

Para asas de baixo alongamento:
Se AR ≤ 4


$a=\dfrac{a_0}{{1+\left(\dfrac{a_0}{\pi \cdot AR}\right)^2} + \dfrac{a_0}{\pi \cdot AR}}$

In [7]:
def low_AR_a(AR, a_0):
    return a_0 \
        /(1 + \
            (a_0/(math.pi * AR)**2 + \
            a_0/(math.pi * AR) \
            )
        )

In [8]:
def a(AR, a_0, tr):
    if AR>4:
        return high_AR_a(AR, a_0, tr)
    if AR <= 4 and AR > 0:
        return low_AR_a(AR, a_0)

a(5, 3, 1)

2.498209815365307